In [3]:
import pandas as pd

df = pd.read_csv("Dataset/corpus_CAT.csv")
df



,title,url,categorie,wikihow,llama2,llm 0,llm -1
0,How to Contact Taco Bell,https://www.wikihow.com/Taco-Bell-Contact,Food and Entertaining,1-800-822-6235This is the number for Taco Bell...,If you have any questions or concerns about ...,"To chop onions without tears, there are seve...","Sure, here are some creative ways to use str..."
1,Don’t Miss Out on These Five Guys Secret Menu ...,https://www.wikihow.com/Five-Guys-Secret-Menu,Food and Entertaining,Your typical burger smothered in extra cheese....,"As an expert in food and entertaining, I hig...",Certainly! Green tea is a popular beverage k...,"Sure, here are some tips on how to cut sideb..."
2,How to Raise Your Spice Tolerance,https://www.wikihow.com/Adapt-to-Spicy-Food,Food and Entertaining,Start out by adding a dash ofmild hot sauceto ...,"Sure, here are some tips on how to raise you...","Sure, here are three quick ways to freeze ba...","Sure, here are some telltale signs that salm..."
3,Enjoy Vegemite,https://www.wikihow.com/Enjoy-Vegemite,Food and Entertaining,Don't eat it plain.Although it may be common p...,"Ah, Vegemite! The quintessential Australian ...",Throwing a surprise party for your mom can b...,Changing a lawn mower blade is a relatively ...
4,Eat Marmite,https://www.wikihow.com/Eat-Marmite,Food and Entertaining,Spread Marmiteverysparingly.In Britain and oth...,Marmite is a popular spread made from yeast ...,"Sure, here's a brief guide on how to dehydra...","Sure, here is a detailed answer on how to re..."
...,...,...,...,...,...,...,...
9395,Make a Temporary Tattoo with Paper,https://www.wikihow.com/Make-a-Temporary-Tatto...,Personal Care and Style,Purchase water slide paper.Most tattoo sticker...,Creating a temporary tattoo using paper is a...,"As an expert in personal care and style, I m...",Determining your neck size and sleeve length...
9396,Make a Temporary Tattoo Last Longer,https://www.wikihow.com/Make-a-Temporary-Tatto...,Personal Care and Style,"Cleanse the spot you plan to tattoo.Lotions, m...",To extend the life of a temporary tattoo bey...,"As an expert in personal care and style, I c...",To stay warm during cold winter nights while...
9397,Create a Sharpie Tattoo,https://www.wikihow.com/Create-a-Sharpie-Tattoo,Personal Care and Style,Draw your tattoo design on your skin.Take your...,Creating a sharpie tattoo is a fun and creat...,Cleaning a cartilage or helix piercing requi...,Painting sheet metal can be a great way to a...
9398,Draw Your Own Temporary Tattoo,https://www.wikihow.com/Draw-Your-Own-Temporar...,Personal Care and Style,Look for inspiration from other people's tatto...,Creating your own temporary tattoo can be a ...,Creating beaded hair barrettes is a fun and ...,"As an expert in personal care and style, I c..."


## Contexte et objectifs du projet MoE

L’objectif est de concevoir un système de classification multi-classe basé sur une architecture de type **Mixture of Experts**. Chaque expert est un **modèle binaire** spécialisé pour détecter une seule classe cible, au sein d’un espace commun. L’approche suit une stratégie One-vs-All, où chaque expert est entraîné à reconnaître une catégorie contre toutes les autres.

Un **classifieur global** est ainsi construit à partir de plusieurs experts indépendants. Ces derniers sont sélectionnés parmi un ensemble d'algorithmes ou modèles (ex : BERT, RoBERTa, etc.), et leur performance est évaluée via des métriques telles que l’**accuracy**, le **F1-score** et l’**EER** (Equal Error Rate).

Un **seuil de décision optimal** peut être défini pour chaque expert en fonction de l’EER. Cela permet d’implémenter un mécanisme de prise de décision.

Pour chaque classe :
- Les données sont **rééquilibrées** (sous-échantillonnage, sur-échantillonnage) pour corriger le déséquilibre du jeu de données.
- Plusieurs modèles peuvent être testés pour une même classe.
- L’algorithme retenu est celui qui montre les **meilleures performances** selon la métrique choisie.
- Une **permutation dynamique des algorithmes** est envisagée selon les résultats.

L’objectif final est de comparer cette approche spécialisée à un classifieur global, et d’identifier les bénéfices en termes de performance, robustesse et contrôle des décisions.





In [6]:
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          Trainer, TrainingArguments)
from torch.utils.data import Dataset

# Paramètres du MoE 
# Ce bloc contient :
#    Un dictionnaire 'classes' qui définit quels modèles LLM sont utilisés comme experts
#    pour chaque classe du problème de classification.
#    Exemple : "Food and Entertaining" est pris en charge par 'bert-base-uncased'.
#
classes = {
    "Food and Entertaining": "bert-base-uncased",
    "Home and Garden": "distilbert-base-uncased",
    "Personal Care and Style": "roberta-base"
}

#    Dataset pytorch

#    Une classe PyTorch personnalisée 'BinaryTextDataset' qui permet de préparer les données
#    pour l'entraînement binaire avec un modèle Transformers.
#    Elle prend une liste de textes, applique la tokenisation via le tokenizer du modèle,
#    et prépare les entrées nécessaires (input_ids, attention_mask, labels) pour le modèle.
#    Cette classe est utilisée pour alimenter le Trainer de HuggingFace qu'on utilise.


class BinaryTextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding="max_length", max_length=64)
        self.labels = labels

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.encodings["input_ids"][idx]),
            "attention_mask": torch.tensor(self.encodings["attention_mask"][idx]),
            "labels": torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)




# Boucle d'entraînement des experts.
# Pour chaque classe du MoE, on prépare un dataset binaire (la classe en 1, le reste en 0).
# Il faut compléter la config du Trainer avec les bons paramètres (epochs, batch, etc),
# instancier le Trainer avec le modèle, le dataset et les args,
# lancer le fine-tuning, puis sauvegarder le modèle entraîné.
# C’est ici qu’on entraîne chaque expert à prendre une décision sur le même espace.

# on sauvegarde chaque expert dedans.
experts = {}
for class_name, model_name in classes.items():
    
    print(f"Training expert for class: {class_name}")

    df_temp = df.copy()
    df_temp["label"] = (df_temp["categorie"] == class_name).astype(int)

    X_train, X_test, y_train, y_test = train_test_split(
        df_temp["title"], df_temp["label"], test_size=0.2, stratify=df_temp["label"], random_state=42)

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    train_dataset = BinaryTextDataset(X_train.tolist(), y_train.tolist(), tokenizer)
    eval_dataset = BinaryTextDataset(X_test.tolist(), y_test.tolist(), tokenizer)

    training_args = TrainingArguments(
    # TODO

        
    )
    # TODO
    # TODO
    pass



# Fonction d’inférence du MoE.
# On applique tous les experts sur le texte et on combine leurs prédictions pour sortir une classe finale.

def moe_predict(text):
    pass
    # TODO

# Utilisation
exemple = "How to cook pasta like a chef"
prediction = moe_predict(exemple)
print(f"Texte : {exemple}\nClasse prédite : {prediction}")


Training expert for class: Food and Entertaining


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() missing 1 required positional argument: 'output_dir'